# Summation & LSTM Model - SNLI Dataset

The following notebook contains the implementation of the baseline summation model, and the standard LSTM model for the SNLI dataset

In [1]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /home/aakashj2412/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

In [3]:
# Function that reads data and parses data from file

def readFileData(filePath):
    with open(filePath, 'r') as f:
        inputRows = [row.split('\t') for row in f.readlines()[1:]]

    inputPremises = [extract(row[1]) for row in inputRows if row[0] in labels]
    inputHypotheses = [extract(row[2]) for row in inputRows if row[0] in labels]
    inputLabels = [labels[row[0]] for row in inputRows if row[0] in labels]
    f.close()

    return [inputPremises, inputHypotheses, inputLabels]

In [5]:
# Reading train and test data

datasetPath = "../../Datasets/SNLI/"
trainData = readFileData(f'{datasetPath}snli_1.0_train.txt')
testData = readFileData(f'{datasetPath}snli_1.0_test.txt')
validationData = readFileData(f'{datasetPath}snli_1.0_dev.txt')

In [6]:
# Hyperparameters

maxLen = 50
epochs = 1000
batchSize = 128
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [7]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [8]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [9]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [10]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [11]:
# Import the GloVe embeddings and generate embeddings dictionary

embeddingsDict = dict()
glovePath = '../../Datasets/GloVe/'
glove = open(f'{glovePath}glove.840B.300d.txt', encoding='utf8')

for line in glove:
    records = line.split()
    word = ''.join(records[:-300])
    vectorDimensions = asarray(records[-300:], dtype='float32')
    embeddingsDict[word] = vectorDimensions

glove.close()

In [12]:
# Iterate through the embeddings and store only those that are present in our vocabulary

embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec

In [13]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

In [14]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

# Alternate between #1 and #2 rnn definitions to switch between summation and LSTM model

rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
# rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

In [15]:
# Apply batch normalization to the two input embeddings separately

premInput = rnn(premInput)
hypoInput = rnn(hypoInput)
premInput = tf.keras.layers.BatchNormalization()(premInput)
hypoInput = tf.keras.layers.BatchNormalization()(hypoInput)

In [16]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
joint = keras.layers.concatenate([premInput, hypoInput])
joint = Dropout(0.2)(joint)
for i in range(3):
    joint = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(joint)
    joint = Dropout(0.2)(joint)
    joint = tf.keras.layers.BatchNormalization()(joint)

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function
pred = Dense(3, activation='softmax')(joint)

In [17]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Summation Model

The following subsection trains the model and reports the findings for the summation model

In [ ]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

In [ ]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

In [ ]:
model.save('../../Models/summation_snli.h5')

In [ ]:
ccc = keras.models.load_model('../../Models/summation_snli.h5')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

We have the following results for the summation model, for the SNLI dataset:

- Training Accuracy: 74%
- Validation Accuracy: 77%
- Test Accuracy: 76%

## LSTM Model

The following subsection trains the model and reports the findings for the LSTM model

In [80]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
4292/4292 [==============================] - 697s 160ms/step - loss: 0.9132 - accuracy: 0.5727 - val_loss: 0.8154 - val_accuracy: 0.6387
Epoch 2/1000
4292/4292 [==============================] - 720s 168ms/step - loss: 0.7878 - accuracy: 0.6549 - val_loss: 0.7282 - val_accuracy: 0.6915
Epoch 3/1000
4292/4292 [==============================] - 735s 171ms/step - loss: 0.7381 - accuracy: 0.6858 - val_loss: 0.6901 - val_accuracy: 0.7122
Epoch 4/1000
4292/4292 [==============================] - 706s 164ms/step - loss: 0.7120 - accuracy: 0.7008 - val_loss: 0.6574 - val_accuracy: 0.7291
Epoch 5/1000
4292/4292 [==============================] - 692s 161ms/step - loss: 0.6956 - accuracy: 0.7095 - val_loss: 0.6401 - val_accuracy: 0.7395
Epoch 6/1000
4292/4292 [==============================] - 694s 162ms/step - loss: 0.6836 - accuracy: 0.7168 - val_loss: 0.6359 - val_accuracy: 0.7431
Epoch 7/1000
4292/4292 [==============================] - 695s 162ms/step - loss: 0.6728 - accuracy:

In [81]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

39/39 [==============================] - 3s 86ms/step - loss: 0.5769 - accuracy: 0.7730
Loss =  0.5768946409225464
Acc =  0.7730048894882202


In [ ]:
model.save('../../Models/LSTM_snli.h5')

In [12]:
ccc = keras.models.load_model('../../Models/LSTM_snli.h5')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

39/39 [==============================] - 4s 78ms/step - loss: 0.5769 - accuracy: 0.7730
0.7730048894882202


We can thus observe the following results for the simple LSTM model, for the SNLI dataset:

- Training Accuracy: 75.82%
- Validation Accuracy: 78.21%
- Test Accuracy: 77.30%